In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def classifier(input_shape, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(16, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=input_shape))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(32, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(64, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
          optimizer='adadelta',
          metrics=['accuracy'])
    return model

def train_classifier(input_shape):
    model = classifier(input_shape, (3, 3, 3), (2, 2, 2))
    model.compile(loss='categorical_crossentropy',
          optimizer=Adam(lr=1e-5),
          metrics=['accuracy'])

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:

def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:10]
    file_list_false = get_dirfiles(output_false)[0:10]
    
    #file_list_true = shuffle(file_list_true)
    #file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-3:]
    file_list_false = get_dirfiles(output_false)[-3:]
    
    #file_list_true = shuffle(file_list_true)
    #file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 20, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = model_20()
        #model = preds3d_dense(48)
        model = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=True)
    
 
    return model

In [5]:
fenlei_fit('Fenge_36_36_36_0615_2', load_check = False, batch_size=128, epochs=100, check_name = 'Fenge_36_36_36_0615')

100%|██████████| 3/3 [00:00<00:00, 2293.64it/s]


Train on 20 samples, validate on 6 samples
Epoch 1/100
20/20 [==============================] - 2s - loss: 0.5629 - acc: 0.7000 - val_loss: 1.0315 - val_acc: 0.5000
Epoch 2/100
20/20 [==============================] - 0s - loss: 0.6788 - acc: 0.7500 - val_loss: 0.7836 - val_acc: 0.6667
Epoch 3/100
20/20 [==============================] - 0s - loss: 1.4080 - acc: 0.6000 - val_loss: 1.5017 - val_acc: 0.5000
Epoch 4/100
20/20 [==============================] - 0s - loss: 0.5146 - acc: 0.6500 - val_loss: 0.7721 - val_acc: 0.5000
Epoch 5/100
20/20 [==============================] - 0s - loss: 0.6335 - acc: 0.8500 - val_loss: 1.4920 - val_acc: 0.5000
Epoch 6/100
20/20 [==============================] - 0s - loss: 0.3386 - acc: 0.8000 - val_loss: 0.8737 - val_acc: 0.5000
Epoch 7/100
20/20 [==============================] - 0s - loss: 0.3199 - acc: 0.9000 - val_loss: 1.3830 - val_acc: 0.5000
Epoch 8/100
20/20 [==============================] - 0s - loss: 0.1959 - acc: 0.9500 - val_loss: 1.1192

In [6]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_36_36_36_0615_2.h5')

In [8]:
#model_pred.load_weights(model_paths + 'Fenge_0613_2.h5')

In [9]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.0


In [10]:
cc

[array([[ 0.70119935,  0.29880065]], dtype=float32),
 array([[ 0.59643644,  0.40356359]], dtype=float32),
 array([[ 0.86713791,  0.13286211]], dtype=float32),
 array([[ 0.67728412,  0.32271591]], dtype=float32),
 array([[ 0.63064045,  0.36935958]], dtype=float32),
 array([[ 0.83028775,  0.16971225]], dtype=float32),
 array([[ 0.1507958 ,  0.84920418]], dtype=float32),
 array([[ 0.61342996,  0.3865701 ]], dtype=float32),
 array([[ 0.04506791,  0.95493215]], dtype=float32),
 array([[ 0.00532997,  0.99466997]], dtype=float32),
 array([[ 0.74624777,  0.25375217]], dtype=float32),
 array([[ 0.53418368,  0.46581638]], dtype=float32),
 array([[ 0.00140637,  0.99859363]], dtype=float32),
 array([[ 0.74789566,  0.25210434]], dtype=float32),
 array([[ 0.57281333,  0.42718664]], dtype=float32),
 array([[ 0.79679275,  0.20320719]], dtype=float32),
 array([[ 0.77939051,  0.2206095 ]], dtype=float32),
 array([[ 0.81540024,  0.1845997 ]], dtype=float32),
 array([[ 0.78190446,  0.21809557]], dtype=flo

In [11]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [12]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.38


In [13]:
cc

[array([[ 0.00177154,  0.99822849]], dtype=float32),
 array([[ 0.00183429,  0.99816579]], dtype=float32),
 array([[ 0.00225548,  0.99774456]], dtype=float32),
 array([[ 0.00486854,  0.99513149]], dtype=float32),
 array([[ 0.00552732,  0.99447268]], dtype=float32),
 array([[ 0.00364602,  0.99635398]], dtype=float32),
 array([[ 0.00273767,  0.99726236]], dtype=float32),
 array([[ 0.0029902 ,  0.99700975]], dtype=float32),
 array([[ 0.00381077,  0.9961893 ]], dtype=float32),
 array([[ 0.03726367,  0.96273637]], dtype=float32),
 array([[ 0.03027943,  0.9697206 ]], dtype=float32),
 array([[ 0.0551643 ,  0.94483572]], dtype=float32),
 array([[ 0.1279466 ,  0.87205344]], dtype=float32),
 array([[ 0.0345425,  0.9654575]], dtype=float32),
 array([[ 0.08410556,  0.91589451]], dtype=float32),
 array([[ 0.08881155,  0.91118848]], dtype=float32),
 array([[ 0.15109502,  0.84890497]], dtype=float32),
 array([[ 0.04773059,  0.95226943]], dtype=float32),
 array([[ 0.5030911,  0.4969089]], dtype=float32

In [14]:
len(file_list_false)

12556

In [15]:
%matplotlib inline
plt.hist(trainX[20].tolist())

NameError: name 'trainX' is not defined

In [ ]:
bb= np.load(file_list_false[0])

In [ ]:
bb.shape